# **Personal Workout Tracker Project with Nutritionix and Sheety APIs**

## **Introduction**

In this project, I set out to build a personalized workout tracker that integrates with two APIs: Nutritionix for retrieving exercise details and Sheety for logging the data into a Google Sheet. The project aimed to deepen my understanding of API interactions, HTTP requests, and secure handling of sensitive data using environment variables.

As I worked through this project, I encountered challenges such as handling complex API responses, managing authentication securely, and working with Python's `datetime` module for accurate time tracking. This notebook documents the process, from the initial setup to the final implementation, highlighting the technical intricacies of each step.

## **Requirements**
- Setup API Credentials and Google Spreadsheet
- Get Exercise Stats with Natural Language Queries
- Setup Google Sheet with Sheety
- Saving Data ino Google Sheets
- Authenticate Sheety API
- Setup Environment Variables

## **Setup Instructions**

Before running this notebook, you need to set up a few things:
**Create a `.env` file**:
- In the root directory of this project (where the notebook is located), create a file named `.env`.
- In this file, you will store your API keys and other sensitive information. The `.env` file should look like this:

   ```plaintext
   ENV_NIX_APP_ID=your_nutritionix_app_id
   ENV_NIX_API_KEY=your_nutritionix_api_key
   ENV_SHEETY_ENDPOINT=your_sheety_endpoint_url
   ENV_SHEETY_USERNAME=your_sheety_username
   ENV_SHEETY_PASSWORD=your_sheety_password

## **Importing Necessary Libraries**

- `datetime`: Provides tools for manipulating dates and times, which are crucial for tracking the workout session.
- `os` and `dotenv`: Used together to securely manage environment varia
- `requests`: Facilitates HTTP requests to interact with external APIs.

In [ ]:
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [ ]:
import os
import requests

## **Loading Environment Variables**

To ensure the security of API keys and other sensitive data, I used environment variables. The `dotenv` library loads these variables from a `.env` file, keeping them separate from the main codebase. This practice not only enhances security but also makes the code more portable, as the `.env` file can be easily configured for different environments.

The environment variables are loaded and stored in specific variables for later use, including API credentials and user-specific data required by Nutritionix.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Personal data for Nutritionix API, which influences calorie calculations.
GENDER = "male"
WEIGHT_KG = 85
HEIGHT_CM = 180
AGE = 35

# Retrieve Nutritionix API credentials from environment variables.
APP_ID = os.getenv("ENV_NIX_APP_ID")
API_KEY = os.getenv("ENV_NIX_API_KEY")

## **Getting User Input**

The input section of the code is designed to collect real-time user data, which includes the type of exercises performed, the start time, and the duration of the workout. This data is critical as it forms the basis of the API query to Nutritionix.

Using the `input()` function, the user's input is captured and stored. The inputs are later used to calculate time intervals and to structure the API request.

In [ ]:
# Prompt the user for exercise details, start time, and duration.
exercise_text = input("Tell me which exercises you did: ")
start_time_input = input("Enter the start time (hh:mm): ")
duration_input = input("Enter the duration (in minutes): ")

## **Processing Date and Time**

In this step, I leveraged the `datetime` module to handle and manipulate time data. The user-provided start time is parsed into a `datetime` object, allowing for precise calculations. 
I then calculated the end time by adding the duration to the start time. 

The `strftime()` method is used to format both the start and end times for consistency, ensuring that the data logged into the Google Sheet is in a standard, readable format.

In [ ]:
# Capture today's date in the required format.
today_date = datetime.now().strftime("%d/%m/%Y")

# Parse the user-provided start time and calculate the end time based on the duration.
start_time = datetime.strptime(start_time_input, "%H:%M")
duration_minutes = float(duration_input)
end_time = start_time + timedelta(minutes=duration_minutes)

# Format the start and end times for display and logging.
start_time_formatted = start_time.strftime("%H:%M")
end_time_formatted = end_time.strftime("%H:%M")

## **Making the Nutritionix API Call**

The core functionality of the project lies in interacting with the Nutritionix API. Here, I constructed an HTTP POST request to the Nutritionix endpoint, passing the exercise query along with user-specific details such as gender, weight, height, and age. These details are vital as they allow the API to return personalized exercise data, such as the number of calories burned.

The `requests` library handles the API call, with the response being stored in JSON format. The challenge here was ensuring the request was correctly structured and authenticated using the API keys retrieved from the environment variables.

In [ ]:
# Construct headers for the API request, including authentication details.
headers = {
    "x-app-id": APP_ID,
    "x-app-key": API_KEY,
}

# Prepare parameters for the POST request, including exercise details and personal data.
parameters = {
    "query": exercise_text,
    "gender": GENDER,
    "weight_kg": WEIGHT_KG,
    "height_cm": HEIGHT_CM,
    "age": AGE
}

# Make the API request and store the response in JSON format.
response = requests.post(exercise_endpoint, json=parameters, headers=headers)
result = response.json()
print(f"Nutritionix API call: \n {result} \n")

## **Logging Data to Google Sheets with Sheety**

The final step involves logging the retrieved exercise data into a Google Sheet via the Sheety API. Each exercise returned by Nutritionix is looped over, and the relevant details are structured into a dictionary that Sheety can process.

The use of a loop ensures that multiple exercises can be logged in a single session. The challenge was correctly formatting the data and handling authentication with Sheety, which required basic authentication using credentials stored in environment variables.

In [ ]:
# Define the Google Sheet name and the Sheety endpoint from environment variables.
GOOGLE_SHEET_NAME = "workout"
sheet_endpoint = os.getenv("ENV_SHEETY_ENDPOINT")

# Iterate over each exercise returned by Nutritionix and log it into the Google Sheet.
for exercise in result["exercises"]:
    # Structure the data in a format accepted by Sheety.
    sheet_inputs = {
        GOOGLE_SHEET_NAME: {
            "date": today_date,
            "time": start_time_formatted,
            "duration": duration_input,
            "exercise": exercise["name"].title(),
            "calories": exercise["nf_calories"]
        }
    }

    # Make the API request to Sheety with basic authentication.
    sheet_response = requests.post(
        sheet_endpoint,
        json=sheet_inputs,
        auth=(
            os.getenv("ENV_SHEETY_USERNAME"),
            os.getenv("ENV_SHEETY_PASSWORD"),
        )
    )

    # Print the response to confirm the data was logged successfully.
    print(f"Sheety Response: \n {sheet_response.text}")

## **Conclusion**

This project was a comprehensive exercise in integrating multiple APIs and managing HTTP requests within Python. The process of securely handling API keys, parsing complex JSON responses, and ensuring the integrity of time-based data were a steep learning curve.

The challenges I encountered, particularly with API authentication and data formatting, involved quite some time spent with debugging and iteration. The result is a functional, automated workout tracker that not only provides instant feedback on exercises but also seamlessly logs this data into a Google Sheet for tracking over time.

Future improvements could include expanding the types of data logged, integrating additional health metrics, or enhancing the user interface for better interactivity.